In [14]:
import zipfile
import numpy as np
import pandas as pd
import re

from collections import defaultdict

## Names dataset for assigning gender

Top 100 US baby names with their gender since 1880

https://www.ssa.gov/oact/babynames/limits.html

In [3]:
zf = zipfile.ZipFile('/data/names.zip')

names = defaultdict(lambda: [0,0])

for name in zf.namelist():
    if name[-4:] == ".txt":
        f = zf.open(name)
        for line in f.readlines():
            elements = line.decode("utf-8").split(",")
            
            current_counts = names[elements[0]]
            
            if elements[1] == 'M':
                names[elements[0]] = [current_counts[0] + int(elements[2]), current_counts[1]]
            else:
                names[elements[0]] = [current_counts[0], current_counts[1] + int(elements[2])]
                                    
zf.close()

In [4]:
def m_or_f(name):
    m = names[name][0]
    f = names[name][1]
    
    if m > f:
        return ['M', m / (m + f)]
    elif f > m:
        return ['F', f / (m + f)]
    else:
        return [None, 0]

## Load salary data

In [31]:
salaries = pd.read_csv('/data/2015-combined-salary-seconded-en.csv')

salaries['Salary Paid'] = salaries['Salary Paid'].apply(lambda x: float(re.sub(r'[\$,]', '', x)))
salaries['gender'] = salaries['First name'].apply( lambda x: m_or_f(x)[0] )


print("Matched", len(salaries) - len(salaries[ salaries['gender'].isnull() ]), "names out of", len(salaries), "entries")

Matched 97634 names out of 115431 entries


In [34]:
salaries.head(1)

,Sector,Last name,First name,Salary Paid,Taxable Benefits,Employer,Job title,Calendar Year,gender
0,Government of Ontario - Ministries,Aniol,Richard,106143.8,$177.35,Aboriginal Affairs,Senior Negotiator / Négociateur principal,2015,M


In [33]:
salaries.groupby(by='gender')['Salary Paid'].mean()

gender
F    123633.893673
M    129244.325020
Name: Salary Paid, dtype: float64

In [47]:
salaries.groupby(by=['gender', 'Sector'])['Salary Paid'].mean()

gender  Sector                                                
F       Colleges                                                  113606.295602
        Crown Agencies                                            133432.128447
        Government of Ontario - Judiciary                         189469.143320
        Government of Ontario - Legislative Assembly & Offices    132821.603238
        Government of Ontario - Ministries                        126845.798845
        Hospitals & Boards of Public Health                       121990.465736
        Municipalities & Services                                 118396.210826
        Ontario Power Generation                                  131934.540558
        Other Public Sector Employers                             127939.435247
        School Boards                                             112720.175094
        Seconded (Children & Youth Services)*                     107269.360000
        Seconded (Community Safety & Correctional Service